## Package Imports

In [1]:
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import uuid
import pandas as pd

from IPython.core.display import display, HTML

## Setup CPD CTL

In [2]:
#DOWNLOAD CPDCTL

PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [5]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.79


In [6]:
CPD_URL = 'https://zen-cpd-zen.cpd-msk-dal10-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud'

In [7]:
! cpdctl config profile set cpd --url {CPD_URL}

In [8]:
#! cpdctl config user set cpd_admin --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}
#! cpdctl config context set cpd --profile cpd --user cpd_user
#! cpdctl config context list
#! cpdctl config context use cpd
#! cpdctl project list --output json

## Get Project Source

In [8]:
PROJECT_ZIP_FILE_NAME = 'RStudioPubmedTopicModeling.zip'

In [9]:
!rm RStudioPubmedTopicModeling.zip
!wget https://github.com/ricdodds/ml101/raw/main/projects/RStudioPubmedTopicModeling.zip
! unzip -l RStudioPubmedTopicModeling.zip

rm: cannot remove 'RStudioPubmedTopicModeling.zip': No such file or directory
--2021-06-17 17:18:19--  https://github.com/ricdodds/ml101/raw/main/projects/RStudioPubmedTopicModeling.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ricdodds/ml101/main/projects/RStudioPubmedTopicModeling.zip [following]
--2021-06-17 17:18:20--  https://raw.githubusercontent.com/ricdodds/ml101/main/projects/RStudioPubmedTopicModeling.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569355 (556K) [application/zip]
Saving to: ‘RStudioPubmedTopicModeling.zip’

RStudioPubmedTopicM 100%[===================>] 556.0

## Create Project for Users

In [10]:
PROJECT_NAME = 'RStudioPubmedTopicModeling'

In [11]:
# GET PROJECTS AND DELETE ANY WITH SAME IMPORT NAME. MAY SWITCH TO CREATE PROJECT WITH NAME + SEQUENCE NUMBER
def delete_project(project_name):
    JMES_QUERY = "resources[?entity.name == '{}'].metadata.guid".format(project_name)
    result = ! cpdctl project list --output json --jmes-query "{JMES_QUERY}"
    project_ids = json.loads(result.s)
    if project_ids:
        for project_id in project_ids:
            print('Deleting project with ID: {}'.format(project_id))
            ! cpdctl project delete --project-id {project_id}

In [12]:
def create_project(project_name):
    STORAGE = {"type": "assetfiles", "guid": str(uuid.uuid4())}
    STORAGE_JSON = json.dumps(STORAGE)
    result = ! cpdctl project create --name {project_name} --output json --raw-output --storage '{STORAGE_JSON}' --jmes-query 'location'
    res_project_id = result.s.split('/')[-1]
    print("Created new '{}' project. ID is: {}".format(project_name, res_project_id))
    return res_project_id

In [13]:
def import_zip_assets(project_id, project_file_name):
    result = ! cpdctl asset import start --project-id {project_id} --import-file {project_file_name} --output json --jmes-query "metadata.id" --raw-output
    import_id = result.s
    print("Import project zip. ID is: {}".format(import_id))
    return import_id

In [14]:
def check_asset_import_state(project_id, import_id):
    ! cpdctl asset import get --project-id {project_id} --import-id {import_id}


In [15]:
def list_project_assets(project_id):
    ! cpdctl asset search --query '*:*' --type-name asset --project-id {project_id}


In [16]:
def validate_project_created(project_name):
    res = ! cpdctl project list --output json
    projects = json.loads(res.s)
    project_found = False
    for project in projects['resources']:
        if project_name == project['entity']['name']:
            project_found = True
            break
    return project_found

In [17]:
df = pd.read_csv('/project_data/data_asset/passwords.csv')
#d = {'username': ['demouser1', 'demouser2'], 'password': ['demouser1', 'demouser2'], 'email': ['jrtorres@us.ibm.com', 'jrtorres@us.ibm.com']}
#df = pd.DataFrame(data=d)
df.head(10)

username           password              email
0  user000    81-none-hole-41  user000@email.com
1  user001  26-count-level-05  user001@email.com
2  user002    07-loss-arms-35  user002@email.com
3  user003   91-brown-king-51  user003@email.com
4  user004  06-shine-clear-60  user004@email.com
5  user005  03-large-horse-92  user005@email.com
6  user006   26-wore-track-17  user006@email.com
7  user007    77-jump-half-46  user007@email.com
8  user008    76-left-shoe-67  user008@email.com
9  user009    99-grew-says-67  user009@email.com

In [18]:
for index, row in df.iterrows():
    user = 'cpd_user_' + str(index)
    context = 'cpd_cont_user_' + str(index)
    ! cpdctl config user set {user} --username {row['username']} --password {row['password']}
    ! cpdctl config context set {context} --profile cpd --user {user}
    ! cpdctl config context use {context}
    delete_project(PROJECT_NAME)
    proj_id = create_project(PROJECT_NAME)
    imp_id = import_zip_assets(proj_id, PROJECT_ZIP_FILE_NAME)
    check_asset_import_state(proj_id, imp_id)
    proj_created = validate_project_created(PROJECT_NAME)
    list_project_assets(proj_id)

Switched to context "cpd_cont_user_0".
Created new 'RStudioPubmedTopicModeling' project. ID is: c9102440-34ee-4fe1-b21e-14ae7a6e8a1a
Import project zip. ID is: 4d852d1f-7151-421f-a8ac-a63e500f9a42
...
              
ID:        4d852d1f-7151-421f-a8ac-a63e500f9a42   
Created:   2021-06-17T17:18:46.015Z   
State:     completed   
...
ID                                     Name                     Created                    Description   Type                     State       Tags   Size   
dae1e3e1-7397-45b0-b8ee-2c9bf059dd3f   RStudio                  2021-06-17T17:18:50.000Z                 hardware_specification   available   []     0   
c63fe4b1-e6ef-44e7-ac86-39a3743f55ff   pubmed_topic_model.Rmd   2021-06-17T17:18:50.000Z                 data_asset               available   []     3705   
f62d62a5-e132-4ead-b982-9e7d254c84c7   RStudio                  2021-06-17T17:18:51.000Z                 environment              available   []     0   
b2308884-269c-481c-bb48-9bee31d27389   pubme

Switched to context "cpd_cont_user_6".
Created new 'RStudioPubmedTopicModeling' project. ID is: cff06e5e-9368-4353-aaaa-1f7860ab21d5
Import project zip. ID is: 1c940d7f-e976-426a-a1c6-8535e40c2cb6
...
              
ID:        1c940d7f-e976-426a-a1c6-8535e40c2cb6   
Created:   2021-06-17T17:20:20.685Z   
State:     completed   
...
ID                                     Name                     Created                    Description   Type                     State       Tags   Size   
d2d42cbe-3d3d-4355-827c-db2db4f2fb00   RStudio                  2021-06-17T17:20:28.000Z                 hardware_specification   available   []     0   
48b4421b-53e5-46fc-bd92-3ce52d5be2e2   pubmed_dataset.rds       2021-06-17T17:20:28.000Z                 data_asset               available   []     535724   
abb9a38e-075e-41ff-9da3-e844eef81bf5   RStudio                  2021-06-17T17:20:29.000Z                 environment              available   []     0   
ab1b60d8-45d1-40ae-9f6a-ade545a6fc78   pub

Switched to context "cpd_cont_user_12".
Created new 'RStudioPubmedTopicModeling' project. ID is: 959cebc8-105e-4c24-a107-0360083a8d2f
Import project zip. ID is: e98eb07f-e4c0-4c0b-a6b7-b0bac85dd3e5
...
              
ID:        e98eb07f-e4c0-4c0b-a6b7-b0bac85dd3e5   
Created:   2021-06-17T17:22:41.576Z   
State:     completed   
...
ID                                     Name                     Created                    Description   Type                     State       Tags   Size   
90508da9-639d-4eaa-95b3-399c0df7823a   RStudio                  2021-06-17T17:22:46.000Z                 hardware_specification   available   []     0   
d1c3a7ef-0291-4100-b192-a13e158e91e7   helpers.R                2021-06-17T17:22:46.000Z                 data_asset               available   []     6632   
00abc5df-706a-4030-9118-0a91ca756ccb   RStudio                  2021-06-17T17:22:48.000Z                 environment              available   []     0   
05f810a2-3e0d-45bf-961e-cd0441002b83   pubm

Switched to context "cpd_cont_user_18".
Created new 'RStudioPubmedTopicModeling' project. ID is: a9b56cf4-40ff-4003-897e-f862005dc4f1
Import project zip. ID is: f175c605-2d73-4292-861f-035ed1f1decb
...
              
ID:        f175c605-2d73-4292-861f-035ed1f1decb   
Created:   2021-06-17T17:25:33.272Z   
State:     completed   
...
ID                                     Name                     Created                    Description   Type                     State       Tags   Size   
5c3bccac-0931-4964-bc49-ac3fcff9dfb6   helpers.R                2021-06-17T17:25:51.000Z                 data_asset               available   []     6632   
67f74809-1044-4dd6-a50b-fe8e969cd743   pubmed_topic_model.Rmd   2021-06-17T17:25:51.000Z                 data_asset               available   []     3705   
73219710-a232-47d7-bfa6-7680e5cac83f   pubmed_dataset.rds       2021-06-17T17:25:51.000Z                 data_asset               available   []     535724   
659ef8a9-00ef-4149-bf45-c1c0fc4cfd3

Switched to context "cpd_cont_user_24".
Created new 'RStudioPubmedTopicModeling' project. ID is: feead3b3-6cbe-4998-8f1a-eef56beddc0e
Import project zip. ID is: 3c6407cc-0b86-4331-9a81-933d8e5e7917
...
              
ID:        3c6407cc-0b86-4331-9a81-933d8e5e7917   
Created:   2021-06-17T17:27:44.065Z   
State:     completed   
...
ID                                     Name                     Created                    Description   Type                     State       Tags   Size   
7adb3bc6-f9a8-4fdb-a9f1-453356446440   RStudio                  2021-06-17T17:28:01.000Z                 hardware_specification   available   []     0   
40c37ead-cc70-46ac-86ad-fc51f50296ba   pubmed_topic_model.Rmd   2021-06-17T17:28:01.000Z                 data_asset               available   []     3705   
5dff1bb7-5e90-4259-9cdd-11233cde56ab   RStudio                  2021-06-17T17:28:02.000Z                 environment              available   []     0   
cedfc4e9-77ae-4768-abb0-976f3be4ab8e   help

## Extra Cleanup Helpers

In [19]:
# REMOVE ALL CONTEXTS
res = ! cpdctl config context list --output json
contexts = json.loads(res.s)
for context in contexts:
    c_name = context['Name']
    print("Unsetting context: {}".format(c_name))
    ! cpdctl config context unset {c_name}
    
! cpdctl config context list

Unsetting context: cpd_cont_user_0
Unsetting context: cpd_cont_user_1
Unsetting context: cpd_cont_user_2
Unsetting context: cpd_cont_user_3
Unsetting context: cpd_cont_user_4
Unsetting context: cpd_cont_user_5
Unsetting context: cpd_cont_user_6
Unsetting context: cpd_cont_user_7
Unsetting context: cpd_cont_user_8
Unsetting context: cpd_cont_user_9
Unsetting context: cpd_cont_user_10
Unsetting context: cpd_cont_user_11
Unsetting context: cpd_cont_user_12
Unsetting context: cpd_cont_user_13
Unsetting context: cpd_cont_user_14
Unsetting context: cpd_cont_user_15
Unsetting context: cpd_cont_user_16
Unsetting context: cpd_cont_user_17
Unsetting context: cpd_cont_user_18
Unsetting context: cpd_cont_user_19
Unsetting context: cpd_cont_user_20
Unsetting context: cpd_cont_user_21
Unsetting context: cpd_cont_user_22
Unsetting context: cpd_cont_user_23
Unsetting context: cpd_cont_user_24
Unsetting context: cpd_cont_user_25
Unsetting context: cpd_cont_user_26
Unsetting context: cpd_cont_user_27
Un

In [26]:
# REMOVE ALL USERS
res = ! cpdctl config user list --output json
users = json.loads(res.s)
for user in users:
    u_name = user['Name']
    print("Unsetting user: {}".format(u_name))
    ! cpdctl config user unset {u_name}
    
! cpdctl config user list 

Unsetting user: cpd_user_0
Unsetting user: cpd_user_1
Unsetting user: cpd_user_2
Unsetting user: cpd_user_3
Unsetting user: cpd_user_4
Unsetting user: cpd_user_5
Unsetting user: cpd_user_6
Unsetting user: cpd_user_7
Unsetting user: cpd_user_8
Unsetting user: cpd_user_9
Unsetting user: cpd_user_10
Unsetting user: cpd_user_11
Unsetting user: cpd_user_12
Unsetting user: cpd_user_13
Unsetting user: cpd_user_14
Unsetting user: cpd_user_15
Unsetting user: cpd_user_16
Unsetting user: cpd_user_17
Unsetting user: cpd_user_18
Unsetting user: cpd_user_19
Unsetting user: cpd_user_20
Unsetting user: cpd_user_21
Unsetting user: cpd_user_22
Unsetting user: cpd_user_23
Unsetting user: cpd_user_24
Unsetting user: cpd_user_25
Unsetting user: cpd_user_26
Unsetting user: cpd_user_27
Unsetting user: cpd_user_28
Unsetting user: cpd_user_29
Nothing to show.
